In [ ]:
import pandas as pd

In [ ]:
%run ../modules/utils.ipynb
%run ../modules/cds.ipynb
%run ../modules/preprocessing.ipynb

%run ../modules/paper_figures.ipynb

In [ ]:
lys_path = '/Users/xyuan/Documents/ChenLab/10Jun2021/degraded/DrChen_tRNA_A_Lys_FA_15.xlsx'
df_wild1 = load_data(lys_path)
lys_path = '/Users/xyuan/Documents/ChenLab/10Jun2021/degraded/16_A_Lys_A.xlsx'
df_alkb1 = load_data(lys_path)
df_wild1.shape, df_alkb1.shape

In [ ]:
lys_path = '/Users/xyuan/Documents/ChenLab/10Jun2021/degraded/21_B_Lys.xlsx'
df_wild2 = load_data(lys_path)
lys_path = '/Users/xyuan/Documents/ChenLab/10Jun2021/degraded/22_B_Lys_A.xlsx'
df_alkb2 = load_data(lys_path)
df_wild2.shape, df_alkb2.shape

In [ ]:
df_wild1 = df_wild1[df_wild1.Mass>1500]
df_alkb1 = df_alkb1[df_alkb1.Mass>1500]
df_wild1.shape, df_alkb1.shape

In [ ]:
df_wild2 = df_wild2[df_wild2.Mass>1500]
df_alkb2 = df_alkb2[df_alkb2.Mass>1500]

In [ ]:
_, df_alkb1_only = diff_dfs(df_wild1, df_alkb1)
df_alkb1_only.shape

In [ ]:
_, df_alkb2_only = diff_dfs(df_wild2, df_alkb2)
df_alkb2_only.shape

In [ ]:
import multiprocessing
from collections import namedtuple
ShiftHit = namedtuple('ShiftHit', 'shift hit')

def func(df_s1, df_s2, shift):
    dfm = match_dfs(df_s1, df_s2, shift=shift)
    sh = ShiftHit(shift=shift, hit=dfm.shape[0])
    return sh

PROCESSES = 8
# params = [(df_wild1, df_alkb1_only, shift) for shift in np.arange(-64, 64, 1)]
params = [(df_wild2, df_alkb2_only, shift) for shift in np.arange(-64, 64, 1)]
with multiprocessing.Pool(PROCESSES) as pool:
    shift_hits = pool.starmap(func, params)

df_shift_hits = pd.DataFrame(shift_hits)    

In [ ]:
fig = px.scatter(df_shift_hits[df_shift_hits.hit>0], x='shift', y='hit', labels={'shift': 'Mass Shift', 'hit': 'Hit Count'})
fig.show()

In [ ]:
# df_sh_1 = df_shift_hits.copy()
# df_sh_1.to_excel('Shift_Lys_1.xlsx')
# df_sh_2 = df_shift_hits.copy()
# df_sh_2.to_excel('Shift_Lys_2.xlsx')

In [ ]:
df_sh_top = df_shift_hits.sort_values('hit', ascending=False).reset_index(drop=True).iloc[:10].copy()

In [ ]:
def func(shift, df_s1, df_s2):
    dfm_l, dfm_r = peer_dfs(df_s1, df_s2, shift=shift)
    df = dfm_l if dfm_l.shape[0] < dfm_r.shape[0] else dfm_r
    bcr = base_calling_random(df)
    return len(bcr[1])

# df_s1 = df_wild1.copy()
# df_s2 = df_alkb1_only.copy()
df_s1 = df_wild2.copy()
df_s2 = df_alkb2_only.copy()
df_sh_top['basecallings'] = df_sh_top.apply(lambda x: func(x['shift'], df_s1, df_s2), axis=1)

In [ ]:
fig = plt.figure(figsize=(2.5, 1.5))
df_plt = df_sh_top.copy().sort_values('shift')
df_plt1 = df_plt.copy()
df_plt1['shift'] = df_plt1['shift'].astype(int)
df_plt1 = df_plt1.set_index('shift')
df_plt1.plot(kind='bar',figsize=(2.5, 1.5))
plt.legend()

fig.tight_layout()
svg_fpath = '/Users/xyuan/Downloads/demo.svg'
plt.savefig(svg_fpath, transparent=True, dpi=300)

In [ ]:
svg_fpath = '/Users/xyuan/Downloads/demo.svg'
draw_df(df_shift_hits[df_shift_hits.hit>0], svg_fpath, x='shift', y='hit')